In [1]:
from datetime import datetime
import pandas as pd

In [2]:
data = pd.read_excel("Data set.xlsx", header=1, parse_dates=False)

In [3]:
def convert_date(dt):
    if dt == "":
        return pd.to_datetime("1991-01-31", "%Y-%m-%d")
    try:
        if dt.startswith("20") and dt[2] != ".":
            if len(dt) < 11:
                return pd.to_datetime(dt, format="%Y-%m-%d %H:%M:%S")
            else:
                return pd.to_datetime(dt, format="%Y-%m-%d")
        elif dt[2] == ".":
            return pd.to_datetime(dt, format="%d.%m.%Y")
    except:
        return pd.to_datetime("1991-01-31", "%Y-%m-%d")

In [4]:
def transform_organization(org: str):
    org = org.strip().replace("\n", " ").replace('"', "").replace("ГБУ", "").replace("Автомобильные дороги", "Автодор").strip()
    return org

In [5]:
def clean_raw_data(data):
    now = datetime.now()
    
    dt = data.copy()
    dt["in_date"] = dt["дата поступления в приют"].astype(str).apply(lambda x: convert_date(x))
    dt["catch_date"] = dt["заказ-наряд дата/ акт о поступлении животного, дата"].astype(str).apply(lambda x: convert_date(x))
    dt.loc[dt["in_date"].isnull(), "in_date"] = dt.loc[dt["in_date"].isnull(), "catch_date"]
    
    dt["is_active"] = data["дата выбытия из приюта"].isnull()
    dt["age"] = now.year - data["возраст, год"].astype(str).str.slice(0, 4).astype(int)
    dt["leave_date"] = data["дата выбытия из приюта"].astype(str).apply(lambda x: convert_date(x))
    
    type_map = {
        "собака": "dog",
        "собака ": "dog",
        "кошка": "cat",
        "Кошка": "cat",
        "Собака": "dog",
    }
    dt["type"] = data["вид "].map(type_map)
    dt["weight"] = data["вес, кг"].fillna(0)
    dt["gender"] = data["пол"].map({"женский": "F", "мужской": "M"})
    dt["size"] = data["размер"].map({
        "малый": "small",
        "средний": "medium",
        "крупный": "big",
        "большой": "big",
    })
    dt["organization"] = dt["эксплуатирующая организация"].apply(lambda x: transform_organization(x))
    dt["district"] = data["административный округ"].str.strip()
    dt["is_social"] = data["Социализировано (да/нет)"] == "да"
    
    avg_dict = dt.loc[dt["is_active"]].groupby(["type", "size"])["weight"].mean().to_dict()
    dt["avg_weight"] = dt[["type", "size"]].apply(lambda a: avg_dict[(a[0], a[1])], axis=1)
    dt["weight_diff"] = dt["weight"] - dt["avg_weight"]
    
    dt["in_date_ym"] = dt["in_date"].dt.to_period("M").astype(str)
    dt["catch_date_ym"] = dt["catch_date"].dt.to_period("M").astype(str)
    
    good_cols = [
        "in_date",
        "is_active",
        "age",
        "type",
        "weight",
        "gender",
        "catch_date",
        "size",
        "organization",
        "district",
        "is_social",
        "leave_date",
        "avg_weight",
        "weight_diff",
        "in_date_ym"
    ]
    
    return dt[good_cols]

In [6]:
df = clean_raw_data(data)

In [7]:
df.head()

,in_date,is_active,age,type,weight,gender,catch_date,size,organization,district,is_social,leave_date,avg_weight,weight_diff,in_date_ym
0,2020-07-29,True,4,dog,25.0,F,2020-07-29,medium,Доринвест,СЗАО,True,NaT,21.111111,3.888889,2020-07
1,2020-07-20,True,10,dog,16.0,F,2020-07-20,medium,Доринвест,СЗАО,False,NaT,21.111111,-5.111111,2020-07
2,2020-03-19,True,1,dog,17.0,F,2020-03-19,medium,Доринвест,ЗелАО,True,NaT,21.111111,-4.111111,2020-03
3,2020-08-10,True,3,dog,28.0,M,2020-08-10,medium,Доринвест,ЗелАО,False,NaT,21.111111,6.888889,2020-08
4,2020-07-29,True,5,dog,25.0,M,2020-07-28,medium,Доринвест,СЗАО,False,NaT,21.111111,3.888889,2020-07


In [8]:
df.to_csv("view_1.csv", index=False)

In [9]:
df["organization"].unique()

array(['Доринвест', 'Автодор ВАО', 'Автодор ЗАО', 'Автодор ЮАО',
       'Автодор СВАО', 'Автодор ЮЗАО', 'Автодор ЮВАО', 'Автодор САО'],
      dtype=object)